In [1]:
from src.kr6 import KR6
import modern_robotics as mr
import numpy as np
from src.tools import*
import sympy as sp

In [2]:
# define the symbolic variables for the joint angles
th1, th2, th3, th4, th5, th6 = sp.symbols("theta_1 theta_2 theta_3 theta_4 theta_5 theta_6")

# Body fixed screw axis matrix with end effector in the wrist center
B =sp.Matrix(  [[0    ,     0,     0,     1,     0,  1],
                [0    ,     1,     1,     0,     1,  0],
                [-1    ,    0,     0,     0,     0,  0],
                [0     ,    0,     0,     0,     0,  0],
                [-0.875,    0,     0,     0,     0,  0],
                [0     ,-0.875,  -0.42,   0,     0,  0]])


# Defining the body Jacobian matrix
Jb = sp.zeros(6,6)
Jb6= B[:,5]
Jb5= Ad(exp6(-B[:,5],th6)) * B[:,4]
Jb4= Ad(exp6(-B[:,5],th6)  * exp6(-B[:,4],th5)) * B[:,3]
Jb3= Ad(exp6(-B[:,5],th6)  * exp6(-B[:,4],th5) * exp6(-B[:,3],th4)) * B[:,2]
Jb2= Ad(exp6(-B[:,5],th6)  * exp6(-B[:,4],th5) * exp6(-B[:,3],th4) * exp6(-B[:,2],th3)) * B[:,1]
Jb1= Ad(exp6(-B[:,5],th6)  * exp6(-B[:,4],th5) * exp6(-B[:,3],th4) * exp6(-B[:,2],th3) * exp6(-B[:,1],th2)) * B[:,0]

# Simplify the expressions
Jb6.simplify()
Jb5.simplify()
Jb4.simplify()
Jb3.simplify()
Jb2.simplify()
Jb1.simplify()
# Contatenate the expressions into a single matrix
Jb[:,0] = Jb1
Jb[:,1] = Jb2
Jb[:,2] = Jb3
Jb[:,3] = Jb4
Jb[:,4] = Jb5
Jb[:,5] = Jb6

# Splitting the matrix into four 3x3 submatrices
Jb_11 = Jb[:3,:3]
Jb_12 = Jb[:3,3:]
Jb_21 = Jb[3:,:3]
Jb_22 = Jb[3:,3:]

In [3]:
Jb_12_det = sp.det(Jb_12)  # defining the determinant of the submatrix Jb_12
Jb_12_det

-sin(theta_5)*sin(theta_6)**2 - sin(theta_5)*cos(theta_6)**2

In [4]:
Jb_12_det_solve = sp.solve(Jb_12_det, th5, th6, dict=True) # Solving the first equation to look at the wrist singularities
Jb_12_det_solve

[{theta_5: 0}, {theta_5: pi}]

In [5]:
Jb_21_det = sp.det(Jb_21) # defining the determinant of the submatrix Jb_21

In [6]:
Jb_21_det_solve = sp.solve(Jb_21_det, th2, th3, th4, dict=True) # Solving the second equation to look at the elbow and shoulder singularities
Jb_21_det_solve

[{theta_2: 3.14159265358979},
 {theta_2: -0.5*I*log(-(13.0*exp(I*theta_3) + 12.0)*exp(-I*theta_3)/(12.0*exp(I*theta_3) + 13.0))},
 {theta_2: -I*log(-sqrt(-(13.0*exp(I*theta_3) + 12.0)*exp(-I*theta_3)/(12.0*exp(I*theta_3) + 13.0)))},
 {theta_2: -I*log(-exp(-I*theta_3))},
 {theta_3: 0.0}]

In [7]:
Jb_21_det_solve[2][th2]

-I*log(-sqrt(-(13.0*exp(I*theta_3) + 12.0)*exp(-I*theta_3)/(12.0*exp(I*theta_3) + 13.0)))

In [8]:
# choosing a value for theta_3 to get solutions for theta_2
theta3_val = -np.pi/4
#Subsituting the value of theta_3 in the equations and converting the solutions to degrees
th2_0 = Jb_21_det_solve[0][th2].evalf()*180/np.pi
th2_1 = Jb_21_det_solve[1][th2].subs(th3, theta3_val).evalf()*180/np.pi
th2_2 = Jb_21_det_solve[2][th2].subs(th3, theta3_val).evalf()*180/np.pi
th2_3 = Jb_21_det_solve[3][th2].subs(th3, theta3_val).evalf()*180/np.pi

print(f"{th2_0:.2f}\n {th2_1:.2f}\n {th2_2:.2f}\n {th2_3:.2f}\n {theta3_val*180/np.pi:.2f}")

180.00
 -68.45
 111.55
 -135.00
 -45.00
